# Transformation des Vokabulars von Propyläum nach skos
## Aufbereiten der CSV-Tabelle

Import statements

In [10]:
import pandas as pd
import numpy as np
from jinja2 import Template

Funktion zum Einlesen einer CSV-Datei.

In [11]:
def read_csv_file(file_path):
    """Reads a CSV file and returns a DataFrame."""
    try:
        df = pd.read_csv(file_path)
        return df
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None
    


Funktion zum Auftrennen eines Strings beim Seperator Strichpunkt.
Die aufgetrennten Teilstrings werden anschließend auf eine Grundform reduziert, die zur Referenzierung im Vokabular verwendet wird.

In [12]:
def split_semicolon_asterisk(value):
    if pd.isna(value) or value is None or value == '':
        return None
    parts = [p.strip() for p in str(value).split(';') if p.strip() != '']
    processed = []
    for p in parts:
        base = p.split('*')[0].strip().lower()
        if 'berufsrollen' in base:
            base = 'berufsrolle'
        processed.append(base)
    if not processed:
        return None
    if len(processed) == 1:
        return processed[0]
    return processed

Das CSV-file wird eingelesen und auf zuvor definierte Funktion auf die zwei Spalten skos:narrower und skos:broader angewendet. 
Für die eine spätere Umwandlung in ein Python-Dictionary, muss das Dataframe um pandas-spezifischen NaN-Werte (not a number) bereinigt werden.
Die Spalten "skos:Concept", "uri", "dct:title" und "skos:related" werden entfernt, da sie für das Erstellen des Vokabulars nicht gebraucht werden.
Abschließend wird das Dataframe transponiert als Grundlage für die weitere Verarbeitung.

In [13]:
df = read_csv_file('Berufsrollen_frueher_A.csv')
df['skos:narrower'] = df['skos:narrower'].apply(split_semicolon_asterisk)
df['skos:broader'] = df['skos:broader'].apply(split_semicolon_asterisk)
df = df.replace({np.nan: None, '' : None})
df = df.iloc[1:] 
df = df.drop(columns=["skos:Concept", "uri", "dct:title", "skos:related"])
df = df.T

Funktion zum Erstellen eines Mappings von Index zu verwendeter Grundform (concept Name)


In [ ]:
def create_index_mapping(df):
    mapping = {}
    i = 0
    for pref_label in df.iloc[0]:
        i += 1
        newindex = i
    #print (newindex)
    newname =pref_label.split("*")[0].lower()
    #print (pref_label, "->", newname)
    mapping[newindex] = newname
    return mapping
    

In [ ]:
wa = create_index_mapping(df)
print(wa)

Austausch der Spaltennamen, sodass pro Spalte ein Concept mit dazugehörigen skos Eigenschaften abgebildet ist.

In [25]:
df1=df.rename(columns=create_index_mapping(df))

['grafiker']


Umwandeln des Dataframes in ein Python-Dictionary

In [19]:
berufsrollen = df1.to_dict()
berufsrollen

{1: {'skos:prefLabel': 'Ägyptologe*in',
  'skos:altLabel': 'Ägyptenforscher*in',
  'dct:description': 'Wissenschaftler*in, der/die das alte Ägypten erforscht.',
  'skos:definition': 'Person, die sich mit der Erforschung der Geschichte, Sprache, Kultur und materiellen Hinterlassenschaften des alten Ägypten beschäftigt.',
  'skos:note': 'Die Ägpytologie hat sich vor der Vor- und Frühgeschichte als eigenes Fach etabliert.',
  'skos:broader': 'berufsrolle',
  'skos:narrower': None},
 2: {'skos:prefLabel': 'Althistoriker*in',
  'skos:altLabel': None,
  'dct:description': 'Historiker*in, der/die sich auf die Alte Geschichte (griechisch-römische Antike) spezialisiert hat.',
  'skos:definition': 'Wissenschaftler*in, die sich mit der politischen, sozialen und kulturellen Geschichte der griechisch-römischen Antike beschäftigt.',
  'skos:note': 'Althistoriker*innen haben sich außer mit Texten oft auch mit materiellen Hinterlassenschaften beschäftigt.',
  'skos:broader': 'historiker',
  'skos:narr

Funktion zum Erstellen einer Liste der Konzepte, die Werte von skos:narrower sind

In [17]:
# make a list of concepts which are value of skos:narrower
def find_narrower_concepts(berufsrollen):
    narrower_concepts = set()
    for concept, attributes in berufsrollen.items():
        narrower = attributes.get('skos:narrower')
        if isinstance(narrower, list):
            narrower_concepts.update(narrower)
        elif isinstance(narrower, str):
            narrower_concepts.add(narrower)
    return narrower_concepts

narrower_concepts = find_narrower_concepts(berufsrollen)

Im Folgenden wird ein Jinja-Template genutzt und unter Verwendung folgender Variablen gerendert: 
+ dem Dictionary, das aus dem Dataframe erstellt wurde,
+ einer Liste von Spaltennamen, die die Verwendung des language qualifiers "@de" regelt und
+ eine Liste der concepte, die hierarchisch den berufsrollen untergeordnet sind.

In [18]:
with open('arch_template.j2') as f:
    tmpl = Template(f.read())

archt_ttl =tmpl.render(trim_blocks=True, lstrip_blocks=True, concepts=berufsrollen, statics = [
    "skos:prefLabel",
    "dct:description",
    "skos:altLabel",
    "skos:definition",
    "skos:note"], narrower_concepts=narrower_concepts)
with open('../arch_leiza.ttl', 'w') as f:
    f.write(archt_ttl)